## PORT MIS 샘플데이터 전처리

### 0. 데이터 불러오기

In [3]:
import pandas as pd
import os
# # 파일 경로
# directory = '../data'

# # 3행까지 제거
# for filename in os.listdir(directory):
#     if filename.startswith("부산_2022") and filename.endswith(".xlsx"):  # 부산_2022시작하는 파일에 대해서
#         file_path = os.path.join(directory, filename)
#         df = pd.read_excel(file_path)  # 엑셀 파일을 데이터프레임으로 읽기
#         df = df.iloc[3:]  # 상단 3행 삭제
#         df.to_excel(file_path, index=False)  # 수정된 데이터프레임을 덮어쓰기

#         print(f"{filename} 삭제 완료.")

In [4]:
import pandas as pd
directory = '../data'

df = pd.read_excel(directory + '/부산_202201.xlsx')
df1 = pd.read_excel(directory + '/부산_202202.xlsx')
df2 = pd.read_excel(directory + '/부산_202203.xlsx')
df3 = pd.read_excel(directory + '/부산_202204.xlsx')
df4 = pd.read_excel(directory + '/부산_202205.xlsx')
df5 = pd.read_excel(directory + '/부산_202206.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '../data/부산_202201.xlsx'

In [ ]:
dfs = pd.read_excel(directory + '/부산_선박입출항현황_2022_상.xlsx')

In [ ]:
dfs = dfs.iloc[11:]  # 상단 3행 삭제 

In [ ]:
dfs.drop(columns = 'z', inplace = True)

In [ ]:
dfs.columns = ['항명', '호출부호', '선박명', '입항횟수_연도', '입항횟수_횟수', '구분', '외내', '입출', '총톤수', '입항일시',
       '출항일시', 'CIQ수속일자', '수리일시', '항해구분', 'MRN 번호', '계선장소_코드', '계선장소_숫자', '계선장소명',
       '차항지', '전출항지', '선박용도', '외항:한국인선원수/내항:해기사선원수', '외항:외국인선원수/내항:보통선원수',
       '승객', '예선', '도선', '부선호출부호1', '부선호출부호2']

In [ ]:
combined_df = pd.concat([df, df1, df2, df3, df4, df5], ignore_index=True)
combined_df.columns = ['순번', '호출부호', '입항횟수_연도', '입항횟수_횟수', '시설사용횟수', '신고톤수', '선박명', '선사/대리점_코드',
       '선사/대리점명', '신청시설_코드', '신청시설_번호', '신청시설명', '신청일시(FROM)',
       '신청일시(TO)', '지정시설_코드', '지정시설_번호', '지정시설명', '지정일시(FROM)',
       '지정일시(TO)', '사용목적명', '예보일시', '허가유무']
combined_df.to_excel(directory + '/부산_combined.xlsx', index=False)

### 1. 컬럼 제거 및 결측치 확인

#### 컬럼 제거(시설허가 : combined_df , 입출항 : dfs)

In [5]:
columns_to_drop = ['신청시설_코드', '신청시설_번호', '신청시설명', '신청일시(FROM)', '신청일시(TO)', '선사/대리점_코드', '선사/대리점명']
combined_df.drop(columns=columns_to_drop, inplace=True)

NameError: name 'combined_df' is not defined

In [6]:
columns_to_drop_dfs = ['외내', 'CIQ수속일자', 'MRN 번호', '차항지', '전출항지','외항:한국인선원수/내항:해기사선원수','외항:외국인선원수/내항:보통선원수', '승객', '예선', '도선', '부선호출부호1', '부선호출부호2']
dfs.drop(columns=columns_to_drop_dfs,inplace=True)

NameError: name 'dfs' is not defined

#### 상세처리

- 입출 : 출항만 사용
- 선박용도 > 최종 전처리시 많은 순으로 고려
- 미허가 행 제외

In [7]:
combined_df = combined_df[combined_df['허가유무'] != '미허가']

NameError: name 'combined_df' is not defined

In [11]:
dfs = dfs[dfs['입출'] == '출항']

### 2. Merge_type 변경 후 Merge

In [12]:
dfs['입항횟수_횟수'] = dfs['입항횟수_횟수'].astype(int)
dfs['입항횟수_연도'] = dfs['입항횟수_연도'].astype(int)
combined_df['입항횟수_횟수'] = combined_df['입항횟수_횟수'].astype(int)
combined_df['입항횟수_연도'] = combined_df['입항횟수_연도'].astype(int)

In [13]:
merged_left = pd.merge(dfs, combined_df, on= ['호출부호','선박명','입항횟수_횟수','입항횟수_연도'], how='left') # 입출항, 시설사용허가 left merge
merged_left

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,시설사용횟수,신고톤수,지정시설_코드,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무
0,부산,041471,제2삼원,2016,1,최종,출항,48,2016-02-01 00:00,2022-04-20 13:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,부산,935218,고려2호,2016,1,최종,출항,50,2016-03-11 15:10,2022-01-26 07:45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,부산,T2FP5,SOKOL,2018,1,최종,출항,486,2018-04-26 00:15,2022-01-24 12:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,부산,3E2043,VOLCANO,2019,35,최종,출항,"1,121",2019-10-30 16:20,2022-01-01 17:50,...,5.0,1.121,WAO,02,O박지 O2 묘지,2022-01-01 09:00,2022-01-03 10:00,출항대기,2022-01-01,허가완료
4,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,49.0,393.0,MKD,01,다대부두 1선석,2022-01-13 18:00,2022-01-28 18:00,수리,2022-01-13,허가완료
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40909,부산,H9HZ,CRANE SOL,2022,4,최종,출항,"4,417",2022-06-30 17:10,2022-06-30 22:00,...,1.0,4.417,WAN,03,남외항 N-3박지,2022-06-30 08:00,2022-06-30 16:00,급유,2022-06-30,허가완료
40910,부산,081791,15대신호,2022,35,최종,출항,149,2022-06-30 17:15,2022-06-30 20:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40911,부산,D7BF,MARIGOLD,2022,2,최종,출항,"7,373",2022-06-30 19:00,2022-06-30 20:30,...,1.0,7.373,WAN,04,남외항 N-4박지,2022-06-30 12:00,2022-06-30 20:00,급유,2022-06-30,허가완료
40912,부산,JG5720,NEW CAMELLIA,2022,137,최종,출항,"19,961",2022-06-30 19:05,2022-06-30 22:20,...,1.0,19.961,MBF,12,신국제여객터미널 12선석,2022-06-30 19:00,2022-06-30 23:00,양적하,2022-06-29,허가완료


#### 중복된 행 제거

In [14]:
# 중복된 행, unique값만 남기기
unique_values = merged_left.drop_duplicates()

In [15]:
unique_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40914 entries, 0 to 40913
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   항명          40914 non-null  object 
 1   호출부호        40914 non-null  object 
 2   선박명         40914 non-null  object 
 3   입항횟수_연도     40914 non-null  int64  
 4   입항횟수_횟수     40914 non-null  int64  
 5   구분          40914 non-null  object 
 6   입출          40914 non-null  object 
 7   총톤수         40894 non-null  object 
 8   입항일시        40914 non-null  object 
 9   출항일시        40914 non-null  object 
 10  수리일시        40889 non-null  object 
 11  항해구분        40894 non-null  object 
 12  계선장소_코드     40894 non-null  object 
 13  계선장소_숫자     40894 non-null  object 
 14  계선장소명       40894 non-null  object 
 15  선박용도        40914 non-null  object 
 16  순번          39827 non-null  float64
 17  시설사용횟수      39827 non-null  float64
 18  신고톤수        39827 non-null  object 
 19  지정시설_코드     39827 non-nul

In [16]:
unique_values.isna().sum() # 계선장소, 예선, 도선, 차항지

항명               0
호출부호             0
선박명              0
입항횟수_연도          0
입항횟수_횟수          0
구분               0
입출               0
총톤수             20
입항일시             0
출항일시             0
수리일시            25
항해구분            20
계선장소_코드         20
계선장소_숫자         20
계선장소명           20
선박용도             0
순번            1087
시설사용횟수        1087
신고톤수          1087
지정시설_코드       1087
지정시설_번호       1087
지정시설명         1087
지정일시(FROM)    1087
지정일시(TO)      1087
사용목적명         1087
예보일시          1087
허가유무          1087
dtype: int64

In [17]:
unique_values.columns

Index(['항명', '호출부호', '선박명', '입항횟수_연도', '입항횟수_횟수', '구분', '입출', '총톤수', '입항일시',
       '출항일시', '수리일시', '항해구분', '계선장소_코드', '계선장소_숫자', '계선장소명', '선박용도', '순번',
       '시설사용횟수', '신고톤수', '지정시설_코드', '지정시설_번호', '지정시설명', '지정일시(FROM)',
       '지정일시(TO)', '사용목적명', '예보일시', '허가유무'],
      dtype='object')

In [18]:
unique_values

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,시설사용횟수,신고톤수,지정시설_코드,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무
0,부산,041471,제2삼원,2016,1,최종,출항,48,2016-02-01 00:00,2022-04-20 13:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,부산,935218,고려2호,2016,1,최종,출항,50,2016-03-11 15:10,2022-01-26 07:45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,부산,T2FP5,SOKOL,2018,1,최종,출항,486,2018-04-26 00:15,2022-01-24 12:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,부산,3E2043,VOLCANO,2019,35,최종,출항,"1,121",2019-10-30 16:20,2022-01-01 17:50,...,5.0,1.121,WAO,02,O박지 O2 묘지,2022-01-01 09:00,2022-01-03 10:00,출항대기,2022-01-01,허가완료
4,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,49.0,393.0,MKD,01,다대부두 1선석,2022-01-13 18:00,2022-01-28 18:00,수리,2022-01-13,허가완료
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40909,부산,H9HZ,CRANE SOL,2022,4,최종,출항,"4,417",2022-06-30 17:10,2022-06-30 22:00,...,1.0,4.417,WAN,03,남외항 N-3박지,2022-06-30 08:00,2022-06-30 16:00,급유,2022-06-30,허가완료
40910,부산,081791,15대신호,2022,35,최종,출항,149,2022-06-30 17:15,2022-06-30 20:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40911,부산,D7BF,MARIGOLD,2022,2,최종,출항,"7,373",2022-06-30 19:00,2022-06-30 20:30,...,1.0,7.373,WAN,04,남외항 N-4박지,2022-06-30 12:00,2022-06-30 20:00,급유,2022-06-30,허가완료
40912,부산,JG5720,NEW CAMELLIA,2022,137,최종,출항,"19,961",2022-06-30 19:05,2022-06-30 22:20,...,1.0,19.961,MBF,12,신국제여객터미널 12선석,2022-06-30 19:00,2022-06-30 23:00,양적하,2022-06-29,허가완료


### 3. 대기시간, 서비스시간 컬럼생성(Entry > Anchor > Berth > Departure)

In [19]:
temp = unique_values.copy()

In [20]:
# 데이터 타입으로 변경 , 잘못된 값 제거(6311 인덱스)
temp['지정일시(TO)'] = pd.to_datetime(temp['지정일시(TO)'], errors='coerce')
temp['지정일시(FROM)'] = pd.to_datetime(temp['지정일시(FROM)'], errors='coerce')

In [21]:
# 접안 대기시간,출항 대기시간 계산
temp['접안_대기시간'] = pd.NaT
temp['출항_대기시간'] = pd.NaT

temp.loc[temp['사용목적명'] == '접안대기', '접안_대기시간'] = temp['지정일시(TO)'] - temp['지정일시(FROM)']
temp.loc[temp['사용목적명'] == '출항대기', '출항_대기시간'] = temp['지정일시(TO)'] - temp['지정일시(FROM)']

In [22]:
temp.head()

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,지정시설_코드,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무,접안_대기시간,출항_대기시간
0,부산,041471,제2삼원,2016,1,최종,출항,48,2016-02-01 00:00,2022-04-20 13:00,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT
1,부산,935218,고려2호,2016,1,최종,출항,50,2016-03-11 15:10,2022-01-26 07:45,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT
2,부산,T2FP5,SOKOL,2018,1,최종,출항,486,2018-04-26 00:15,2022-01-24 12:00,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT
3,부산,3E2043,VOLCANO,2019,35,최종,출항,"1,121",2019-10-30 16:20,2022-01-01 17:50,...,WAO,02,O박지 O2 묘지,2022-01-01 09:00:00,2022-01-03 10:00:00,출항대기,2022-01-01,허가완료,NaT,2 days 01:00:00
4,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,MKD,01,다대부두 1선석,2022-01-13 18:00:00,2022-01-28 18:00:00,수리,2022-01-13,허가완료,NaT,NaT


#### Departure 생성

- 출항대기시간 null 값인 경우에는, Departure 값이 '출항일시' 값
- 사용목적명이 '출항대기'일 경우에 Departure 값이 '신청일시(FROM)' 값으로

In [23]:
import numpy as np

# 'Service_Time_End' 열 생성
temp['Service_Time_End'] = np.nan

# '사용목적명'이 '출항대기'인 경우 'Service_Time_End' 값 할당
temp.loc[temp['사용목적명'] == '출항대기', 'Service_Time_End'] = temp['지정일시(FROM)']

# '출항_대기시간'이 null인 경우 'Service_Time_End' 값 할당
temp.loc[temp['출항_대기시간'].isnull(),'Service_Time_End'] = temp['출항일시']

In [24]:
temp.head(15)

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,지정시설_번호,지정시설명,지정일시(FROM),지정일시(TO),사용목적명,예보일시,허가유무,접안_대기시간,출항_대기시간,Service_Time_End
0,부산,041471,제2삼원,2016,1,최종,출항,48,2016-02-01 00:00,2022-04-20 13:00,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,2022-04-20 13:00
1,부산,935218,고려2호,2016,1,최종,출항,50,2016-03-11 15:10,2022-01-26 07:45,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,2022-01-26 07:45
2,부산,T2FP5,SOKOL,2018,1,최종,출항,486,2018-04-26 00:15,2022-01-24 12:00,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,2022-01-24 12:00
3,부산,3E2043,VOLCANO,2019,35,최종,출항,"1,121",2019-10-30 16:20,2022-01-01 17:50,...,02,O박지 O2 묘지,2022-01-01 09:00:00,2022-01-03 10:00:00,출항대기,2022-01-01,허가완료,NaT,2 days 01:00:00,2022-01-01 09:00:00
4,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,01,다대부두 1선석,2022-01-13 18:00:00,2022-01-28 18:00:00,수리,2022-01-13,허가완료,NaT,NaT,2022-06-01 06:30
5,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,01,다대부두 1선석,2022-01-28 18:00:00,2022-02-28 18:00:00,기타,2022-01-18,허가완료,NaT,NaT,2022-06-01 06:30
6,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,01,다대부두 1선석,2022-02-28 18:00:00,2022-03-31 18:00:00,수리,2022-02-25,허가완료,NaT,NaT,2022-06-01 06:30
7,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,01,감천 1부두 1선석,2022-04-05 13:00:00,2022-04-30 18:00:00,수리,2022-04-01,허가완료,NaT,NaT,2022-06-01 06:30
8,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,01,감천 1부두 1선석,2022-04-30 18:00:00,2022-05-15 18:00:00,상가대기,2022-04-29,허가완료,NaT,NaT,2022-06-01 06:30
9,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,01,감천 1부두 1선석,2022-05-13 08:00:00,2022-05-31 18:00:00,수리,2022-05-12,허가완료,NaT,NaT,2022-06-01 06:30


#### Anchor = 접안대기

In [25]:
temp['Anchor'] = np.nan

# '사용목적명'이 '출항대기'인 경우 'Anchor' 값 할당
temp.loc[temp['사용목적명'] == '접안대기', 'Anchor'] = temp['지정일시(FROM)']

In [26]:
temp['Service_Time_Start'] = np.where(temp['Anchor'].isnull(), temp['입항일시'], temp['Anchor'])

In [27]:
# 문자열을 datetime으로 변환
temp['Service_Time_End'] = pd.to_datetime(temp['Service_Time_End'])
temp['Service_Time_Start'] = pd.to_datetime(temp['Service_Time_Start'])
temp['Service_Time'] = temp['Service_Time_End'] - temp['Service_Time_Start']

In [28]:
temp

,항명,호출부호,선박명,입항횟수_연도,입항횟수_횟수,구분,입출,총톤수,입항일시,출항일시,...,지정일시(TO),사용목적명,예보일시,허가유무,접안_대기시간,출항_대기시간,Service_Time_End,Anchor,Service_Time_Start,Service_Time
0,부산,041471,제2삼원,2016,1,최종,출항,48,2016-02-01 00:00,2022-04-20 13:00,...,NaT,NaN,NaN,NaN,NaN,NaT,2022-04-20 13:00:00,NaN,2016-02-01 00:00:00,2270 days 13:00:00
1,부산,935218,고려2호,2016,1,최종,출항,50,2016-03-11 15:10,2022-01-26 07:45,...,NaT,NaN,NaN,NaN,NaN,NaT,2022-01-26 07:45:00,NaN,2016-03-11 15:10:00,2146 days 16:35:00
2,부산,T2FP5,SOKOL,2018,1,최종,출항,486,2018-04-26 00:15,2022-01-24 12:00,...,NaT,NaN,NaN,NaN,NaN,NaT,2022-01-24 12:00:00,NaN,2018-04-26 00:15:00,1369 days 11:45:00
3,부산,3E2043,VOLCANO,2019,35,최종,출항,"1,121",2019-10-30 16:20,2022-01-01 17:50,...,2022-01-03 10:00:00,출항대기,2022-01-01,허가완료,NaN,2 days 01:00:00,2022-01-01 09:00:00,NaN,2019-10-30 16:20:00,793 days 16:40:00
4,부산,JVWQ7,GERMES,2020,1,최종,출항,498,2020-01-02 07:00,2022-06-01 06:30,...,2022-01-28 18:00:00,수리,2022-01-13,허가완료,NaN,NaT,2022-06-01 06:30:00,NaN,2020-01-02 07:00:00,880 days 23:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40909,부산,H9HZ,CRANE SOL,2022,4,최종,출항,"4,417",2022-06-30 17:10,2022-06-30 22:00,...,2022-06-30 16:00:00,급유,2022-06-30,허가완료,NaN,NaN,2022-06-30 22:00:00,NaN,2022-06-30 17:10:00,0 days 04:50:00
40910,부산,081791,15대신호,2022,35,최종,출항,149,2022-06-30 17:15,2022-06-30 20:30,...,NaT,NaN,NaN,NaN,NaN,NaN,2022-06-30 20:30:00,NaN,2022-06-30 17:15:00,0 days 03:15:00
40911,부산,D7BF,MARIGOLD,2022,2,최종,출항,"7,373",2022-06-30 19:00,2022-06-30 20:30,...,2022-06-30 20:00:00,급유,2022-06-30,허가완료,NaN,NaN,2022-06-30 20:30:00,NaN,2022-06-30 19:00:00,0 days 01:30:00
40912,부산,JG5720,NEW CAMELLIA,2022,137,최종,출항,"19,961",2022-06-30 19:05,2022-06-30 22:20,...,2022-06-30 23:00:00,양적하,2022-06-29,허가완료,NaN,NaN,2022-06-30 22:20:00,NaN,2022-06-30 19:05:00,0 days 03:15:00


### 서비스타임,대기율, 용도별 대기율(컨테이터/비컨테이너)

In [29]:
waiting = temp.copy()

In [30]:
waiting['접안_대기시간'].fillna(0, inplace = True)

In [31]:
# import pandas as pd

# # 데이터프레임에서 '접안_대기시간'과 'Service_Time' 컬럼을 timedelta 형식으로 변환
waiting['접안_대기시간'] = pd.to_timedelta(waiting['접안_대기시간'])
waiting['Service_Time'] = pd.to_timedelta(waiting['Service_Time'])

# timedelta 값에서 일(day)로 변환
waiting['접안_대기시간_NumDays'] = waiting['접안_대기시간'].dt.days
waiting['Service_Time_NumDays'] = waiting['Service_Time'].dt.days

In [32]:
# '접안_대기시간' 컬럼에서 'days', 'hours', 'minutes'를 분리하여 새로운 열로 추가
waiting['접안_대기시간_일'] = waiting['접안_대기시간'].dt.days
waiting['접안_대기시간_시간'] = waiting['접안_대기시간'].dt.components.hours
waiting['접안_대기시간_분'] = waiting['접안_대기시간'].dt.components.minutes
# 'Service_Time' 컬럼에서 'days', 'hours', 'minutes'를 분리하여 새로운 열로 추가
waiting['Service_Time_일'] = waiting['Service_Time'].dt.days
waiting['Service_Time_시간'] = waiting['Service_Time'].dt.components.hours
waiting['Service_Time_분'] = waiting['Service_Time'].dt.components.minutes

# 'Service_Time'을 분으로 변환하여 'Service_Time_분' 열에 추가
waiting['Service_Time_분'] = waiting['Service_Time_일'] * 24 * 60 + waiting['Service_Time_시간'] * 60 + waiting['Service_Time_분']

# '접안_대기시간'을 분으로 변환하여 '접안_대기시간_분' 열에 추가
waiting['접안_대기시간_분'] = waiting['접안_대기시간_일'] * 24 * 60 + waiting['접안_대기시간_시간'] * 60 + waiting['접안_대기시간_분']

In [33]:
waiting['대기율'] = waiting['접안_대기시간_분'] / waiting['Service_Time_분']

In [34]:
waiting['대기율'].describe()

count    4.083800e+04
mean              inf
std               NaN
min     -6.545455e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max               inf
Name: 대기율, dtype: float64

In [35]:
waiting['대기율'].fillna(0, inplace = True)

In [36]:
waiting['대기율'].mean()

inf

In [2]:
waiting['대기율'].describe()

NameError: name 'waiting' is not defined

In [40]:
waiting.columns

Index(['항명', '호출부호', '선박명', '입항횟수_연도', '입항횟수_횟수', '구분', '입출', '총톤수', '입항일시',
       '출항일시', '수리일시', '항해구분', '계선장소_코드', '계선장소_숫자', '계선장소명', '선박용도', '순번',
       '시설사용횟수', '신고톤수', '지정시설_코드', '지정시설_번호', '지정시설명', '지정일시(FROM)',
       '지정일시(TO)', '사용목적명', '예보일시', '허가유무', '접안_대기시간', '출항_대기시간',
       'Service_Time_End', 'Anchor', 'Service_Time_Start', 'Service_Time',
       '접안_대기시간_NumDays', 'Service_Time_NumDays', '접안_대기시간_일', '접안_대기시간_시간',
       '접안_대기시간_분', 'Service_Time_일', 'Service_Time_시간', 'Service_Time_분',
       '대기율'],
      dtype='object')

In [48]:
result = waiting.groupby('선박용도')['대기율'].mean().to_frame()
result = result.sort_values(by='대기율', ascending=False).reset_index()

In [49]:
result

,선박용도,대기율
0,철강재 운반선,0.121411
1,LPG 운반선,0.101338
2,냉동.냉장선,0.065231
3,일반화물선,0.061611
4,풀컨테이너선,0.042494
5,압항 예선,0.039296
6,석유제품 운반선,0.037223
7,산물선(벌크선),0.029535
8,케미칼 운반선,0.028879
9,세미(혼재)컨테이너선,0.020692


In [50]:
waiting.to_csv(directory + '부산_전처리ver1.csv', encoding = 'utf-8-sig', index = False)